In [17]:
torch.nn.DataParallel?

In [1]:
import sys
sys.path.append('../')
import os
import torch
import random
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from collections import defaultdict
from sklearn.model_selection import train_test_split
import pydicom
os.environ['CUDA_VISIBLE_DEVICES']="None"
%matplotlib inline

## saving npy files

In [ ]:
arr = [1,3,4,5]

In [ ]:
np.save('data/test.npy', np.array(arr).astype(np.float32))

In [ ]:
arr_l = np.load('data/test.npy')

In [ ]:
arr_l

In [ ]:
os.path.exists('data')

In [ ]:
fnames = np.load('data/npy_data/train_fnames.npy')

In [ ]:
fnames.shape

## tesing train test split bug

In [ ]:
df = pd.read_csv("data/stage_1_train_labels.csv")
class_df = pd.read_csv('data/stage_1_detailed_class_info.csv')
# df = df[df["Target"] == 0]
# df.reset_index(drop=True, inplace=True)
# df_groups = df.groupby(["patientId"]).groups
# fnames = list(df_groups.keys())


In [ ]:
class_df.head(10)

In [ ]:
df.head()

In [ ]:
# df['class'] = df.apply(lambda x: class_df[class_df['patientId'] == x]['class'])
df['class'] = class_df['class']

In [ ]:
df.head(10)

In [ ]:
image_df = df.groupby('patientId').apply(lambda x: x.sample(1))

In [ ]:
image_df.head(10)

In [ ]:
image_df.sample?

In [ ]:
image_df.shape

In [ ]:
raw_train_df, valid_df = train_test_split(image_df, test_size=0.1, random_state=2018, stratify=image_df['class'])
print(raw_train_df.shape, 'training data')
print(valid_df.shape, 'validation data')

In [ ]:
raw_train_df.head()

In [ ]:
raw_train_df['class'].value_counts()

In [ ]:
valid_df['class'].value_counts()

In [ ]:
tf = list(raw_train_df.patientId)
vf = list(valid_df.patientId)

In [ ]:
len(tf), len(vf)

In [ ]:
tf[:4]

In [ ]:
'''Check if tf and vf are mutually exclusive'''
print(len(tf + vf) == image_df.shape[0])
len(set(tf + vf)) == len(tf) + len(vf)

In [ ]:
len(tf), len(vf)

##### sanity check

In [ ]:
train_fnames = np.load('data/npy_data/train_fnames.npy')
val_fnames = np.load('data/npy_data/val_fnames.npy')

In [ ]:
len(train_fnames), len(val_fnames), (len(set(list(train_fnames) + list(val_fnames))) == len(train_fnames) + len(val_fnames))

In [ ]:
tl = np.load('data/npy_data/train_labels.npy')
vl = np.load('data/npy_data/val_labels.npy')

In [ ]:
np.unique(tl, return_counts=True)

In [ ]:
np.unique(vl, return_counts=True)

## class analysis of the boxes

In [ ]:
class_df = pd.read_csv('data/stage_1_detailed_class_info.csv')

In [ ]:
class_df.head()

In [ ]:
class_df.groupby('class').groups

In [ ]:
class_df.head(10)()

In [ ]:
train_df = pd.read_csv('data/stage_1_train_labels.csv')
train_df.head(10)

In [ ]:
train_df.shape, class_df.shape

In [ ]:
grp = class_df.groupby(["patientId"]).groups

In [ ]:
class_df.iloc[12790]['class']

In [ ]:
class_df['class'].nunique()

In [ ]:
class_df.groupby(["class"]).nunique()

## Analysing anchor boxes (priors)

In [ ]:
from layers import PriorBox
from data.config import cfg
    

In [ ]:
priors = PriorBox(cfg).forward() * 300

In [ ]:
5776, 2166, 600, 150, 36, 4

In [ ]:
canvas = np.ones([300, 300])
colors = ['b', 'g', 'r', 'c', 'm', 'y', 'k']
count = 0
fig, ax = plt.subplots(figsize=(10, 10))
T = 100
for idx, box in enumerate(priors):
    x, y, w, h = box
    XMin = x - w
    YMin = y - h
    if count<=T:
#         print(XMin, YMin)
#         print(x,y, w,h)
        rect = plt.Rectangle((XMin, YMin), width=w, height=h, edgecolor=random.choice(colors), fill=None, linewidth=2)
        ax.add_patch(rect)
#         fig.canvas.draw();
#     if count==T: break
    count += 1
print(count)
ax.imshow(canvas, cmap='binary')
plt.show();

In [ ]:
def area(box):
    XMin, YMin, XMax, YMax = box
    w = XMax - XMin + 1
    h = YMax - YMin + 1
#     print(abs(w), abs(h))
    return abs(w*h)

In [ ]:
for idx, box in enumerate(priors):
#     print(box)
    a = area(box)
    print(a)
#     break

In [ ]:
priors


## random testing

## Testing data

In [ ]:
df = pd.read_csv('data/stage_1_train_labels.csv')

In [ ]:
df.head()

In [ ]:
df = df[df['Target'] == 1]
df_groups = df.groupby(["patientId"]).groups

In [ ]:
df.head()

In [ ]:
df.iloc[0]

In [ ]:
random.seed(69)
df_groups = df.groupby(["patientId"]).groups
fnames = df_groups.keys()

In [ ]:

# train_fnames, val_fnames = train_test_split(fnames, test_size=0.2)
# fnames = train_fnames if phase == "train" else val_fnames
boxes = []
labels = []
for name in fnames:
    indices = df_groups[name]
    box, label = [], []
    for idx in indices:
        line = df.iloc[idx]
        if line['Target']:
            x, y = line["x"], line["y"]
            box.append([x, y, (x + line["width"]), (y + line["height"])])
    labels.append(np.zeros(len(box)))
    boxes.append(box)
    if len(boxes) >10: break  # just for testing
num_samples = len(fnames)


In [ ]:
len(fnames)

## Testing dataloader

In [2]:
from data.dataloader import provider
# from utils.augmentations import *

HOME: /home/eee/ug/15084015/kaggle/rsna-pneumonia-detection-challenge
DATA_ROOT:  /home/eee/ug/15084015/kaggle/rsna-pneumonia-detection-challenge/data


In [3]:
df = pd.read_csv("../data/stage_1_train_labels.csv")
df_groups = df.groupby(["patientId"]).groups

In [4]:
dataloader = provider(phase='train', num_workers=4)

In [7]:
# dataloader = provider(phase='train', num_workers=0)
# size=300
# mean=(104, 117, 123)
# transforms = [
#     ConvertFromInts(),
#     PhotometricDistort(),
#     Expand(mean),
# #     RandomSampleCrop(),
#     RandomMirror(),
#     ToPercentCoords(),
#     Resize(300),
#     SubtractMeans(mean)
# ]

# dataloader.dataset.transform.augment = Compose(transforms)

In [5]:
fnames, images, boxes = next(iter(dataloader))
index_range = iter(range(len(images)))

/home/eee/ug/15084015/miniconda3/envs/ML/lib/python3.7/site-packages/imgaug/imgaug.py:2064: RuntimeWarning: invalid value encountered in sqrt
  distance_maps = np.sqrt(distance_maps)
/home/eee/ug/15084015/miniconda3/envs/ML/lib/python3.7/site-packages/imgaug/imgaug.py:2064: RuntimeWarning: invalid value encountered in sqrt
  distance_maps = np.sqrt(distance_maps)
/home/eee/ug/15084015/miniconda3/envs/ML/lib/python3.7/site-packages/imgaug/imgaug.py:2064: RuntimeWarning: invalid value encountered in sqrt
  distance_maps = np.sqrt(distance_maps)
/home/eee/ug/15084015/miniconda3/envs/ML/lib/python3.7/site-packages/imgaug/imgaug.py:2064: RuntimeWarning: invalid value encountered in sqrt
  distance_maps = np.sqrt(distance_maps)
Process Process-3:
Traceback (most recent call last):
  File "../utils/aug_imgaug.py", line 464, in __call__
    aug_bbs = self.seq_det.augment_bounding_boxes([bbs])[0]
  File "/home/eee/ug/15084015/miniconda3/envs/ML/lib/python3.7/site-packages/imgaug/augmenters/meta

In [6]:
boxes

[tensor([[ 0.,  0.,  0.,  0., -1.]]),
 tensor([[0.2031, 0.4424, 0.3643, 0.5664, 0.0000]]),
 tensor([[0.3018, 0.2168, 0.5205, 0.5820, 0.0000],
         [0.6504, 0.3574, 0.8633, 0.6865, 0.0000]]),
 tensor([[ 0.,  0.,  0.,  0., -1.]]),
 tensor([[0.1582, 0.4688, 0.4551, 0.7363, 0.0000],
         [0.5938, 0.4863, 0.9004, 0.7656, 0.0000]]),
 tensor([[ 0.,  0.,  0.,  0., -1.]]),
 tensor([[ 0.,  0.,  0.,  0., -1.]]),
 tensor([[0.6123, 0.2920, 0.8750, 0.6582, 0.0000],
         [0.2080, 0.4385, 0.5107, 0.6094, 0.0000]])]

In [9]:
# images[i].shape
# images[i].numpy().transpose(1,2,0) + np.array((104, 117, 123), dtype=np.float32)

In [16]:
'''Plot random images from the dataloader with the original ground truth bboxes'''
# i = random.choice(list(range(len(images))))
i = next(index_range)
print(i, boxes[i])
fig = plt.figure(figsize=(40, 10))
ax = fig.add_subplot(111)
j = 0
for box in boxes[i]:
    box = [i.item() * 300 for i in box[:4]]
#     print(box)
    XMin, YMin, XMax, YMax = box
    w = XMax - XMin
    h = YMax - YMin
    rect = plt.Rectangle((XMin, YMin), width=w, height=h, edgecolor='r', fill=None, linewidth=2)
    ax.add_patch(rect)
    j += 1
img_aug = images[i].numpy().transpose(1,2,0) + np.array((104, 117, 123), dtype=np.float32)
img_aug = img_aug.astype('int')
ax.imshow(img_aug, cmap="gist_gray", label="aug image")


ax = fig.add_subplot(122)
# print(fnames[i])
gt_boxes = []
indices = df_groups[fnames[i]]
for idx in indices:
    line = df.iloc[idx]
    x, y = line["x"], line["y"]
    if line["Target"]:
        gt_boxes.append([x, y, (x + line["width"]), (y + line["height"])])
img = pydicom.read_file('../data/stage_1_train_images/' + fnames[i] + ".dcm")
img = np.expand_dims(img.pixel_array, -1)
img = img.repeat(3, axis=-1)
for box in gt_boxes:
    XMin, YMin, XMax, YMax = box
    w = XMax - XMin
    h = YMax - YMin
    rect = plt.Rectangle((XMin, YMin), width=w, height=h, edgecolor='r', fill=None, linewidth=2)
    ax.add_patch(rect)
    j += 1
ax.imshow(img / 255, cmap="gist_gray", label="org image")
plt.show()

StopIteration: 

In [16]:
iaa.Affine?

In [ ]:
np.max(images[i].numpy().transpose(1,2,0))

In [ ]:
import imgaug as ia
from imgaug import augmenters as iaa

ia.seed(1)


In [ ]:
ia.BoundingBoxesOnImage?

In [ ]:

image = ia.quokka(size=(256, 256))
bbs = ia.BoundingBoxesOnImage([ia.BoundingBox(*[0, 0, 1, 1])], shape=image.shape)

seq = iaa.Sequential([
    iaa.Multiply((1.2, 1.5)), # change brightness, doesn't affect BBs
    iaa.Affine(
        translate_px={"x": 40, "y": 60},
        scale=(0.5, 0.7)
    ) # translate by 40/60px on x/y axis, and scale to 50-70%, affects BBs
])

seq_det = seq.to_deterministic()
image_aug = seq_det.augment_images([image])[0]
bbs_aug = seq_det.augment_bounding_boxes([bbs])[0]

# print coordinates before/after augmentation (see below)
# use .x1_int, .y_int, ... to get integer coordinates
for i in range(len(bbs.bounding_boxes)):
    before = bbs.bounding_boxes[i]
    after = bbs_aug.bounding_boxes[i]
    print("BB %d: (%.4f, %.4f, %.4f, %.4f) -> (%.4f, %.4f, %.4f, %.4f)" % (
        i,
        before.x1, before.y1, before.x2, before.y2,
        after.x1, after.y1, after.x2, after.y2)
    )

# image with BBs before/after augmentation (shown below)
image_before = bbs.draw_on_image(image, thickness=2)
image_after = bbs_aug.draw_on_image(image_aug, thickness=2, color=[0, 0, 255])


In [ ]:
bbs_aug.

In [ ]:
plt.imshow(image_after)

## Prepare VOC formatted dataset

In [ ]:
from pascal_voc_writer import Writer
import random
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from collections import defaultdict
%matplotlib inline

In [ ]:
root = '/home/eee/ug/15084015/ML/codefest/data/'
image_data_root = '/home/eee/ug/15084015/ML/codefest/data/image_data/'

In [ ]:
'''Prepare xml annotations using bbox_train.csv file'''

image_root = 'VOCFace/JPEGImages/'
annotations_root = 'VOCFace/Annotations/'

df = pd.read_csv(root + "train.csv")
box_df = pd.read_csv(root + "bbox_train.csv")
fnames = df["Name"].values
box_df_groups = box_df.groupby(["Name"]).groups

for name in tqdm(fnames):
    indices = box_df_groups[name]
    width = box_df.iloc[indices[0]]['width']
    height = box_df.iloc[indices[0]]['height']
    writer = Writer(image_root + '17182.jpg', width, height)
    for idx in indices:
        line = box_df.iloc[idx]
        writer.addObject('face', line["xmin"], line["ymin"], line["xmax"], line["ymax"])
    writer.save(annotations_root + name.split('.')[0] + '.xml')

In [ ]:
'''Create trainval.txt file too '''
fnames_clean = [x.split('.')[0] for x in fnames]
trainvalfile = open('VOCFace/ImageSets/Main/trainval.txt', 'w')
for item in fnames_clean:
    trainvalfile.write("%s\n" % item)